In [ ]:
#Neetakshi Sharma Assignment 2 
#Titanic w/ Kaggle

#data analysis tools 

import pandas as pd
import numpy as np 
import random as rnd

#visualization tools 
import seaborn as sns
import matplotlib.pyplot as plt

#machine learning tools 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

#acquiring data set

train_df=pd.read_csv("/Users/Catarina/Dropbox/Paulo - Python,R,SQL,Netlogo - Data Science\Python/titanic_survival_machine_learn/train.csv")

test_df=pd.read_csv("/Users/Catarina/Dropbox/Paulo - Python,R,SQL,Netlogo - Data Science\Python/titanic_survival_machine_learn/test.csv")

combine =[train_df,test_df]


#Descriptive summary

#train_df.info()

#test_df.info()

print(train_df.columns.values) # nome

#print (train_df.describe())

#print (test_df.describe())

#print (train_df.describe(include=['O']))


#Primary correlations 

#print (train_df[["Pclass" , "Survived"]].groupby(["Pclass"], as_index=False).mean().sort_values(by="Survived",ascending=False))


#print (train_df[["Sex" , "Survived"]].groupby(["Sex"], as_index=False).mean().sort_values(by="Survived",ascending=False))

#print (train_df[["SibSp" , "Survived"]].groupby(["SibSp"], as_index=False).mean().sort_values(by="Survived",ascending=False))


#Primary visualization data 

#histogram grouped by age x survived

'''
g=sns.FacetGrid(data=train_df,col="Survived")
g.map(plt.hist,"Age", bins=20)
'''

#histogram grid grouped by age x class x survived

'''
grid=sns.FacetGrid(train_df,col="Survived",row="Pclass",size=2.2,aspect=1.6)
grid.map(plt.hist,"Age",alpha=0.5,bins=20)
grid.add_legend()
'''
#histogram grouped by sex x class x survived

'''
grid=sns.FacetGrid(train_df,row="Embarked",size=2.2, aspect=1.6)
grid.map(sns.pointplot,"Pclass","Survived","Sex",palette="deep") 
#Pclass = Row / Survived = Col and Sex = category division
grid.add_legend()
'''

#histogram correlating fare x embark x survived

'''
grid=sns.FacetGrid(train_df,row="Embarked",col="Survived")
grid.map(sns.barplot,"Sex","Fare",alpha=0.5,ci=None)
'''



#Data wrangling

#dropping features

'''
print ("Before",train_df.shape,test_df.shape,combine[0].shape,combine[1].shape)
'''
train_df=train_df.drop(["Ticket","Cabin"],axis=1)

test_df=test_df.drop(["Ticket","Cabin"],axis=1)

combine=[train_df,test_df]
'''
print ("After",train_df.shape,test_df.shape)
'''
#Extracting names title

for dataset in combine:
	dataset["Title"]=dataset.Name.str.extract("([A-Za-z]+)\.",expand=False)

#print(pd.crosstab(train_df["Title"],train_df["Sex"]))


for dataset in combine:
	dataset["Title"]=dataset["Title"].replace(["Lady","Countess","Capt","Col","Don","Major","Rev","Sir","Jonkheer","Dona"],"Rare")

	dataset["Title"]=dataset["Title"].replace("Mlle","Miss")
	dataset["Title"]=dataset["Title"].replace("Ms","Miss")
	dataset["Title"]=dataset["Title"].replace("Mme","Mrs")

title_mapping={"Mr":1,"Miss":2,"Mrs":3,"Master":4,"Rare":5}

for dataset in combine:
	dataset["Title"]=dataset["Title"].map(title_mapping)
	dataset["Title"]=dataset["Title"].fillna(0)

#dropping no more necessary variables

train_df=train_df.drop(["Name","PassengerId"],axis=1)
test_df=test_df.drop(["Name"],axis=1)

combine=[train_df,test_df]

#print (train_df.shape,test_df.shape)

#converting sex = string to gender = numeric value to adequate input model

for dataset in combine:
	dataset["Sex"]=dataset["Sex"].map({"female":1,"male":0}).astype(int)

'''
print (train_df.head(2))
grid=sns.FacetGrid(train_df,row="Pclass",col="Sex",size=2.2,aspect=1.6)
grid.map(plt.hist,"Age",alpha=0.5,bins=20)
grid.add_legend()
plt.show()
'''

#print ("Before",train_df.head())

guess_age=np.zeros((2,3))

#print (guess_age)

for dataset in combine:
	for i in range(0,2):
		for j in range(0,3):
			guess_df=dataset[(dataset["Sex"]==i) & (dataset["Pclass"] == j+1)]["Age"].dropna()

			age_guess=guess_df.median()
			guess_age[i,j]= int(age_guess / 0.5 + 0.5) * 0.5

	for i in range(0,2):
		for j in range (0,3):
			dataset.loc[(dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),"Age"] = guess_age[i,j]

	dataset["Age"] = dataset["Age"].astype(int)

#print ("After",train_df.head())

#creating age bands for survived correlation 

train_df["Ageband"]=pd.cut(train_df["Age"],5) #script para corte

#print(train_df[["Ageband","Survived"]].groupby(["Ageband"],as_index=False).mean().sort_values(by="Ageband",ascending=True))

for dataset in combine:
	dataset.loc[dataset["Age"] <=16,"Age"]=0
	dataset.loc[(dataset["Age"] > 16) & (dataset["Age"]<=32),"Age"]=1
	dataset.loc[(dataset["Age"] > 33) & (dataset["Age"]<=48),"Age"]=2
	dataset.loc[(dataset["Age"] > 48) & (dataset["Age"]<=64),"Age"]=3
	dataset.loc[dataset["Age"] > 64, "Age"]=4

#print (train_df.head())

train_df=train_df.drop(["Ageband"],axis=1)
combine=[train_df,test_df]
#print(train_df.head(2))

#Create new variable FamilySize - Combines Parch x SibSp

for dataset in combine:
	dataset["FamilySize"] = dataset ["SibSp"] + dataset["Parch"] + 1

#agrupamento por tamanho da familia + survived

#print(train_df[["FamilySize","Survived"]].groupby(["FamilySize"],as_index=False).mean().sort_values(by="FamilySize",ascending=False)) 

for dataset in combine: #agrupando quem está sozinh (1) ou não
	dataset["IsAlone"]=0
	dataset.loc[dataset["FamilySize"]==1,"IsAlone"]=1

#print(train_df[["IsAlone","Survived"]].groupby(["IsAlone"],as_index=False).mean())

train_df=train_df.drop(["Parch","SibSp","FamilySize"],axis=1)
test_df=test_df.drop(["Parch","SibSp","FamilySize"],axis=1)
combine=[train_df,test_df]

#print(train_df.head())

for dataset in combine:
	dataset["Age*Class"] = dataset.Age * dataset.Pclass

#print (train_df.loc[:,["Age*Class","Age","Pclass"]].head(10))

#print(train_df.columns.values) # nome

#Embarked x Survived

freq_port=train_df.Embarked.dropna().mode()[0]

for dataset in combine: 
	dataset["Embarked"] = dataset["Embarked"].fillna(freq_port)

# print (train_df[["Embarked","Survived"]].groupby(["Embarked"],as_index=False).mean().sort_values(by="Survived",ascending=False)) #print sobre embarked x survived

for dataset in combine:
	dataset["Embarked"] = dataset["Embarked"].map({"S":0,"C":1,"Q":2}).astype(int)

#print (train_df.head())

#Quick numeric completing of Fare variable values

#tirando NA e colocando valores de NA
test_df["Fare"].fillna(test_df["Fare"].dropna().median(),inplace=True)

#Creating Fare bands

train_df["FareBand"]=pd.cut(train_df["Fare"],4)

(train_df[["FareBand","Survived"]].groupby(["FareBand"],as_index=False).mean().sort_values(by="FareBand",ascending=True))

for dataset in combine: 
	dataset.loc[dataset["Fare"] <= 7.91,"Fare"]=0
	dataset.loc[(dataset["Fare"] > 7.91) & (dataset["Fare"] <= 14.454),"Fare"]=1
	dataset.loc[(dataset["Fare"] > 14.454) & (dataset["Fare"] <= 31),"Fare"]=2
	dataset.loc[dataset["Fare"] > 31,"Fare"]=3
	dataset["Fare"] = dataset["Fare"]

train_df = train_df.drop(["FareBand"],axis=1)
combine = [train_df,test_df]

#print ("Test", test_df.columns.values)

#print ("Train",train_df.columns.values)

#"Model, prediction and solving"

'''
Logistic Regression
KNN or k-Nearest Neighbors
Support Vector Machines
Naive Bayes classifier
Decision Tree
Random Forrest
Perceptron
Artificial neural network
RVM or Relevance Vector Machine
'''

X_train=train_df.drop("Survived",axis=1)
Y_train=train_df["Survived"]
X_test=test_df.drop("PassengerId",axis=1).copy()

#print(X_train.shape,Y_train.shape,X_test.shape)

#Logistic model 

logreg = LogisticRegression() #modelo
logreg.fit(X_train,Y_train) #treino do modelo sendo (X,Y)
Y_pred = logreg.predict(X_test) #teste com o modelo 
acc_log = round(logreg.score(X_train,Y_train) * 100,2)
print (acc_log)

coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.collumns = ["Features"]
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])

#print ("Logistic regression",coeff_df.sort_values(by="Correlation",ascending=False))


#Support vector machine

svc=SVC()
svc.fit(X_train,Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train,Y_train) * 100, 2 )

print ("SVC - support vector",acc_svc)


#KNN - K Nearest Neighbors Algorithm

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train,Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train,Y_train) * 100, 2)

print ("Knn",acc_knn)

#Gaussian Naive Bayes 

gaussian = GaussianNB()
gaussian.fit(X_train,Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train,Y_train) * 100,2)

print ("Naive bayes",acc_gaussian)


#Perceptron 

perceptron = Perceptron()
perceptron.fit(X_train,Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train,Y_train)*100,2)

print ("Perceptron",acc_perceptron)

#Linear Support Vector Machine  LSVC

linear_svc = LinearSVC()
linear_svc.fit(X_train,Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train,Y_train) * 100,2)

print ("Acc_linear_svc",acc_linear_svc)

#Stochastic Gradiant Descent

sgd = SGDClassifier()
sgd.fit(X_train,Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train,Y_train) * 100,2)

print ("Sgd",acc_sgd)


#Decision tree

dtree = DecisionTreeClassifier()
dtree.fit(X_train,Y_train)
Y_pred = dtree.predict(X_test)
acc_dtree = round(dtree.score(X_train,Y_train) * 100,2)

print ("Dtree",acc_dtree)


#Random decision tree

rdtree = RandomForestClassifier()
rdtree.fit(X_train,Y_train)
Y_pred = rdtree.predict(X_test)
acc_rdtree = round(rdtree.score(X_train,Y_train) * 100,2)

print ("RDtree",acc_rdtree)


#Model score classification

models = pd.DataFrame(
	{"Model":
	
	['Support Vector Machines', 'KNN','Logistic Regression','Random Forest','Naive Bayes', 'Perceptron', "Stochastic Gradient Decent",'Linear SVC', "Decision Tree"],

	"Score": [acc_svc, acc_knn, acc_log, 
              acc_rdtree, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_dtree]

	 })

print ("Rank of models",models.sort_values(by="Score",ascending=False))
